In [1]:
from transformers import (DataCollatorForTokenClassification,
                          BertTokenizer, BertForTokenClassification)
from transformers import Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
pretrained_model = "./Model_01.01/04/model/"
dataset = DatasetDict().load_from_disk('./built_datasets/peykareh.01')
model = BertForTokenClassification.from_pretrained(pretrained_model, num_labels=3)
print(model)
print(dataset)

In [2]:

trainer = Trainer(model=model,
                  args=TrainingArguments(output_dir='./Model/results'),
                  train_dataset=dataset['train'],
                  eval_dataset=dataset["test"])
print(dataset["test"])
print(dataset["test"][0])
print(type(dataset["test"]))
print(type(dataset["test"][0]))


In [4]:
predictions = trainer.predict(dataset["test"])
print(predictions)
predicted_labels = predictions.predictions.argmax(axis=-1)
print(predicted_labels)

In [10]:
print(dataset['test'])

In [5]:
from labeler import Evaluator
from corpus_processor import Type
evaluator = Evaluator(labels=(0,1,2))
evaluator.evaluate(dataset['test']['labels'],predicted_labels,Type.sents_raw)

In [6]:
evaluator.show_metrics()

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

predictions_flat = [label for sample in predicted_labels for label in sample]
true_labels_flat = [label for sample in dataset['test']['labels'] for label in sample]

# Calculate the accuracy, precision, recall, and F1 score
accuracy = accuracy_score(true_labels_flat, predictions_flat)
precision = precision_score(true_labels_flat, predictions_flat, average='macro')
recall = recall_score(true_labels_flat, predictions_flat, average='macro')
f1 = f1_score(true_labels_flat, predictions_flat, average='macro')

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 score: {f1:.2f}")


In [10]:
report = classification_report(true_labels_flat, predictions_flat,digits=5)

# Print the report to the console
print(report)